In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime as dt
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import time


In [18]:
import nbclean

In [16]:
"""
delete Notebook outputs
"""


def delete_notebook_outputs(file_path):
	c = nbclean.clean.NotebookCleaner(file_path)
	c.clear('output')
	c.save(file_path)


delete_notebook_outputs('/Users/higashi/PycharmProjects/WebScraping/venv/weibo/weibo_scraping.ipynb')


In [8]:
"""
arg: url
return: html
"""


def get_feed_html_soup(driver, url):
    retries = 3
    i = 0
    
    while i < retries:
        try:
            driver.get(url)
            html = driver.page_source.encode('utf-8')
            html_soup = BeautifulSoup(html, "html.parser")
            
            return html_soup
        except TimeoutException:
            i += 1
            print('Timeout, Retrying... {} / {}'.format((i+1), retries))
            
    return None


In [4]:
"""
arg: html_soup
return: mid_list
"""


def get_mid_list(html_soup):
    mid_list = []
    div_elements = html_soup.find_all('div', {'id': 'pl_feedlist_index'})
    div_elements = div_elements[0].find_all('div', {'class': 'card-wrap'})
    for elem in div_elements:
        mid = elem.get('mid')
        if type(mid) == str:
            mid_list.append(mid)
    
    return mid_list


In [39]:
"""
arg: mid
return: account_name, feed_time, sentence, share, comment, good
"""


def get_feed_info(html_soup, mid):
    div_mid = html_soup.find_all('div', {'mid': mid})
    
    # アカウント名
    account_name = div_mid[0].find('p').get('nick-name')
    
    # 時刻
    feed_time = ''.join(div_mid[0].find_all('p', {'class': 'from'})[0].find('a').text.split())
    try:
        if '今天' in feed_time:     
            feed_time = dt.strftime(dt.now(), '%Y-%m-%d ') + feed_time.split('今天')[1] + ':00'
        elif '前' in feed_time:
            feed_time = dt.strftime(dt.now(), '%Y-%m-%d %H:%M:%S')
        elif '秒' in feed_time:
            feed_time = dt.strftime(dt.now(), '%Y-%m-%d %H:%M:%S')
        elif len(feed_time) < 12:
            feed_time = dt.strptime(feed_time, '%m月%d日%H:%M').replace(year=2019)
        else:
            feed_time = dt.strptime(feed_time, '%Y年%m月%d日%H:%M')
    except:
        feed_time = 'not detected'
    # 本文
    sentence = ''.join(div_mid[0].find('p').text.split())
    
    # シェア、コメント、いいね
    share = ''
    comment = ''
    suda_data = ''
    good = ''
    bottom_items = div_mid[0].find_all('div', {'class': 'card-act'})[0].find_all('li')
    for bottom_item in bottom_items:
        
        if bottom_item.text == '收藏':
            pass
        elif '转发' in bottom_item.text:
            if len(bottom_item.text.split()) < 2:
                share = '0'
            else:
                share = bottom_item.text.split()[1]
        elif '评论' in bottom_item.text:
            if len(bottom_item.text.split()) < 2:
                comment = '0'
            else:
                comment = bottom_item.text.split()[1]
                suda_data = bottom_item.find_all('a')[0].get('suda-data')
        else:
            if len(bottom_item.text.split()) < 1:
                good = '0'
            else:
                good = bottom_item.text   
    
    return account_name, feed_time, sentence, share, comment, suda_data, good


In [6]:
"""
arg: html_soup
return: comment_id_list
"""


def get_comment_id_list(html_soup):
    comment_id_list = []
    div_elements = html_soup.select('div')

    for elem in div_elements:
        comment_id = elem.get('comment_id')
        if type(comment_id) == str:
            comment_id_list.append(comment_id)
    
    return comment_id_list


In [7]:
"""
arg: comment_id
return: comment_account_name, comment_text
"""


def get_comment_name_sentence(html_soup, comment_id):
    div_comment = html_soup.find_all('div', {'comment_id': comment_id})
    comment_account_name = div_comment[0].find('div', {'class': 'txt'}).text.split()[0]
    comment_sentence = ''
    if len(div_comment[0].find('div', {'class': 'txt'}).text.split()) == 3:
        comment_sentence = div_comment[0].find('div', {'class': 'txt'}).text.split()[2]
    elif len(div_comment[0].find('div', {'class': 'txt'}).text.split()) == 2:
        comment_sentence = 'None or emoji'
    return comment_account_name, comment_sentence


In [20]:
"""
arg: html_soup, mid
return: account_link
"""


def get_account_link(html_soup, mid):
    div_mid = html_soup.find_all('div', {'mid': mid})
    time.sleep(1)
    account_link = div_mid[0].find_all('a', {'class': 'name'})[0].get('href')
    
    return account_link


In [9]:
"""
arg: account_link
return: html_soup
"""


def get_account_html_soup(driver, account_link):
    driver.get(account_link)
    time.sleep(5)  # 3sだとデータが取りきれない場合がある
    
    html = driver.page_source.encode('utf-8')
    html_soup = BeautifulSoup(html, "html.parser")
    
    return html_soup


In [10]:
"""
arg: html_soup
return: rank, location, gender, follow, follower, weibo, get_time
"""


def get_account_info(html_soup):
    rank = get_rank(html_soup)
    location = get_location(html_soup)
    gender = get_gender(html_soup)
    follow, follower, weibo = get_follow_follower_weibo(html_soup)
    get_time = get_now_time()
    
    return rank, location, gender, follow, follower, weibo, get_time


In [11]:
"""
arg: html_Soup
return: rank
"""


def get_rank(html_soup):
    span_all = html_soup.find_all('span')

    rank = ''

    for span_tag in span_all:
        if "Lv" in span_tag.text:
            rank = span_tag.text
        
    if rank == '':
        rank = 'no rank'
        
    return rank


In [12]:
"""
arg: html_soup
return: location
"""


def get_location(html_soup):
    if html_soup.find_all('em', {'class': 'W_ficon ficon_cd_place S_ficon'}):
        span = html_soup.find_all('span', {'class': 'item_text W_fl'})

        location = ''

        for tag in span:
            if 'Lv' in tag.text:
                location = span[1].text.split()
                break
            else:
                location = span[0].text.split()

        if type(location) == list:
            word_concat = ''
            for word in location:
                word_concat += word
            location = word_concat
    else:
        location = 'no location'
        
    return location


In [13]:
"""
arg: html_soup
return: gender
"""


def get_gender(html_soup):

    if html_soup.find_all('i', {'class': 'W_icon icon_pf_male'}):
        gender = 'male'
    elif html_soup.find_all('i', {'class': 'W_icon icon_pf_female'}):
        gender = 'female'
    else:
        gender = 'no gender'
        
    return gender


In [41]:
"""
arg: html_soup
return: follow, follower, weibo
"""


def get_follow_follower_weibo(html_soup):
    if html_soup.find_all('strong'):
        strongs = html_soup.find_all('strong')
        try:
            follow = strongs[0].text
        except:
            follow = 'no follow'
        
        try:
            follower = strongs[1].text
        except:
            follower = 'no follower'
        
        try:
            weibo = strongs[2].text
        except:
            weibo = 'no weibo'
    else:
        follow = 'no follow'
        follower = 'no follower'
        weibo = 'no weibo'
        
    return follow, follower, weibo


In [15]:
"""
arg: None
return: now_time
"""


def get_now_time():
    now = dt.strftime(dt.now(), '%Y-%m-%d %H:%M:%S')
    
    return now


In [16]:
"""
arg: None
return: driver
"""


def set_driver():
    options = Options()

    # options.set_headless(True)
    options.binary_location = "/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary"
    # options.add_argument("--headless")

    chromedriver_path = "/Users/higashi/Desktop/Document/chromedriver/chromedriver"

    return webdriver.Chrome(options=options, executable_path=chromedriver_path)


In [28]:
"""
arg: url
return: urls
"""


def get_feed_links(start_url):
    urls = []
    urls.append(start_url)
    
    url = start_url
    
    for i in range(50):  # 何ページ目まで読むか
        r = requests.get(url)
        html_contents = r.text

        html_soup = BeautifulSoup(html_contents)
        try:
            next_link = 'https://s.weibo.com' + html_soup.find_all('a', {'class': 'next'})[0].get('href')
            if next_link in urls:
                break
            else:
                urls.append(next_link)
                url = next_link
        except:
            break

    return urls


In [18]:
"""
arg: driver
return: elems, elems_suda_data
"""


def get_comment_button_list(driver):
    elems = []
    elems_suda_data = []

    element = driver.find_elements_by_tag_name('a')
    for elem in element:
        val_string = elem.get_attribute('action-type')
        try:
            if val_string == 'feed_list_comment':
                elems.append(elem)
                elems_suda_data.append(elem.get_attribute('suda-data'))
        except:
            pass
    
    return elems, elems_suda_data


In [19]:
"""
arg: suda_data, elems, elems_suda_data
return: None
"""


def click_comment(suda_data, elems, elems_suda_data):
    try:
        for i, elem in enumerate(elems):
            if suda_data == elems_suda_data[i]:
                element = elem
                break
            else:
                continue
        element.click()
        time.sleep(5)  # 2s程度待たないとcomment欄が表示されない(javascript実行時間)
    except:
        pass


In [56]:
"""
テスト1
"""

# driver起動
TIMEOUT = 10
driver = set_driver()
driver.set_page_load_timeout(TIMEOUT)


In [57]:
"""
テスト2
"""


# ログインのために一度読み込む
url = 'https://s.weibo.com/weibo?q=toyota&wvr=6&b=1&display=0&retcode=6102&Refer=SWeibo_box'
driver.get(url)

html = driver.page_source.encode('utf-8')
html_soup = BeautifulSoup(html, "html.parser")


In [58]:
"""
テスト3
"""

# リストを準備
account_names = []
feed_times = []
sentences = []
shares = []
comments = []
goods = []
ranks = []
locations = []
genders = []
follows = []
followers = []
weiboes = []
get_times = []
types = []

comment_account_names = []
comment_sentences = []
comment_types = []
# コメントについては、下記情報は一旦保留
comment_locations = []
comment_genders = []
comment_ranks = []
comment_follows = []
comment_followers = []
comment_weiboes = []
comment_post_times = []
comment_get_times = []
comment_shares = []
comment_comments = []
comment_goods = []

# コメント取得実装のため一時的
# count = 0

# スタートリンクの設定
start_url = 'https://s.weibo.com/weibo?q=toyota&wvr=6&b=1&display=0&retcode=6102&Refer=SWeibo_box'

# フィードのリンクを全て取得
feed_links = get_feed_links(start_url)
print('getting feed links is done.')
print('{} links are detected.'.format(len(feed_links)))

# フィードページごとにデータ取得
for i, feed_link in enumerate(feed_links):
    print('progress... : {} / {} feed link'.format(i, len(feed_links)))
    # フィードのhtmlを取得
    feed_html_soup = get_feed_html_soup(driver, feed_link)
    print('getting feed html_soup is done.')
    
    # commentボタンリストを取得
    # elems, elems_suda_data = get_comment_button_list(driver)
    
    # midリストを取得
    if feed_html_soup:
        mids = get_mid_list(feed_html_soup)
        print('getting mid_list is done.')
        print('{} mids are detected.'.format(len(mids)))
    else:
        continue
    # midごとにデータ取得
    for j, mid in enumerate(mids):
        print('    progress... : {} / {} mid'.format(j, len(mids)))
        # アカウントリンクを取得
        account_link = 'https:' + get_account_link(feed_html_soup, mid)
        print('    getting account_link is done.')
        
        # コメント取得実装のため一時的
        # if count == 1:
            # break
        
        # フィードデータを取得
        account_name, feed_time, sentence, share, comment, suda_data, good = get_feed_info(feed_html_soup, mid)
        print('    getting feed info is done')
        
        """
        # コメントが存在する場合、コメントを取得しに行く
        if comment != '0':
            click_comment(suda_data, elems, elems_suda_data)
            print("clicked")
            html = driver.page_source.encode('utf-8')
            html_soup = BeautifulSoup(html, "html.parser")
            comment_id_list = get_comment_id_list(html_soup)
            print(comment_id_list)
            
            global comment_account_names, comment_sentences, comment_types, comment_locations
            global comment_genders, comment_ranks, comment_follows, comment_followers
            global comment_weiboes, comment_post_times, comment_get_times, comment_shares
            global comment_comments, comment_goods
            comment_account_names.clear()
            comment_sentences.clear()
            comment_types.clear()
            # コメントについては、下記情報は一旦保留
            comment_locations.clear()
            comment_genders.clear()
            comment_ranks.clear()
            comment_follows.clear()
            comment_followers.clear()
            comment_weiboes.clear()
            comment_post_times.clear()
            comment_get_times.clear()
            comment_shares.clear()
            comment_comments.clear()
            comment_goods.clear()
            
            for comment_id in comment_id_list:
                comment_account_name, comment_sentence = get_comment_name_sentence(html_soup, comment_id)
                comment_account_names.append(comment_account_name)
                comment_sentences.append(comment_sentence)
                comment_types.append('comment')
                # コメントについては、下記情報は一旦保留
                comment_locations.append('None')
                comment_genders.append('None')
                comment_ranks.append('None')
                comment_follows.append('None')
                comment_followers.append('None')
                comment_weiboes.append('None')
                comment_post_times.append('None')
                comment_get_times.append('None')
                comment_shares.append('None')
                comment_comments.append('None')
                comment_goods.append('None')
        """
        
        # アカウント詳細のhtmlを取得
        account_html_soup = get_account_html_soup(driver, account_link)
        print('    getting account html_soup is done.')
        
        # アカウントデータを取得
        rank, location, gender, follow, follower, weibo, get_time = get_account_info(account_html_soup)
        print('    getting account info is done.')
        
        # タイプを定義
        type_name = 'feed'
        
        # フィードデータ格納
        account_names.append(account_name)
        feed_times.append(feed_time)
        sentences.append(sentence)
        shares.append(share)
        comments.append(comment)
        goods.append(good)
        ranks.append(rank)
        locations.append(location)
        genders.append(gender)
        follows.append(follow)
        followers.append(follower)
        weiboes.append(weibo)
        get_times.append(get_time)
        types.append(type_name)
        
        """
        # フィードとコメントを連結
        account_names = account_names + comment_account_names
        feed_times = feed_times + comment_post_times
        sentences = sentences + comment_sentences
        shares = shares + comment_shares
        comments = comments + comment_comments
        goods = goods + comment_goods
        ranks = ranks + comment_ranks
        locations = locations + comment_locations
        genders = genders + comment_genders
        follows = follows + comment_follows
        followers = followers + comment_followers
        weiboes = weiboes + comment_weiboes
        get_times = get_times + comment_get_times
        types = types + comment_types
        """
        
        print('    storing data is done')
        
        # コメント取得実装のため一時的
        # count = count + 1

# csv保存
os.chdir('/Users/higashi/PycharmProjects/WeiboScraping/venv/data')

df = pd.DataFrame({
    'Account': account_names,
    'Location': locations,
    'Gender': genders,
    'Rank': ranks,
    'Follow': follows,
    'Follower': followers,
    'Weibo': weiboes,
    'FeedTime': feed_times,
    'GetTime': get_times,
    'Type': types,
    'Sentence': sentences,
    'Share': shares,
    'Comment': comments,
    'Good': goods
})

df.to_csv('data_tmp' + '.csv')


getting feed links is done.
38 links are detected.
progress... : 0 / 38 feed link


getting feed html_soup is done.
getting mid_list is done.
23 mids are detected.
    progress... : 0 / 23 mid


    getting account_link is done.
    getting feed info is done


    getting account html_soup is done.
    getting account info is done.
    storing data is done
    progress... : 1 / 23 mid


    getting account_link is done.
    getting feed info is done


...

    getting account_link is done.
    getting feed info is done


    getting account html_soup is done.
    getting account info is done.
    storing data is done
    progress... : 19 / 20 mid


    getting account_link is done.
    getting feed info is done


    getting account html_soup is done.
    getting account info is done.
    storing data is done
progress... : 27 / 38 feed link


TimeoutException: Message: timeout
  (Session info: chrome=74.0.3726.0)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.14.1 x86_64)


In [60]:
os.chdir('/Users/higashi/PycharmProjects/WeiboScraping/venv/data')

df = pd.DataFrame({
    'Account': account_names,
    'Location': locations,
    'Gender': genders,
    'Rank': ranks,
    'Follow': follows,
    'Follower': followers,
    'Weibo': weiboes,
    'FeedTime': feed_times,
    'GetTime': get_times,
    'Type': types,
    'Sentence': sentences,
    'Share': shares,
    'Comment': comments,
    'Good': goods
})

df.to_csv('data_tmp' + '.csv')


In [59]:
print(len(account_names))
print(len(locations))
print(len(genders))
print(len(ranks))
print(len(follows))
print(len(followers))
print(len(weiboes))
print(len(feed_times))
print(len(get_times))
print(len(sentences))
print(len(shares))
print(len(comments))
print(len(goods))

539
539
539
539
539
539
539
539
539
539
539
539
539


In [179]:
driver = set_driver()
url = 'https://s.weibo.com/weibo/%25E7%25A4%25BE%25E4%25BC%259A?topnav=1&wvr=6&b=1&display=0&retcode=6102'
driver.get(url)

html = driver.page_source.encode('utf-8')
html_soup = BeautifulSoup(html, "html.parser")

In [202]:
start_url = 'https://s.weibo.com/weibo/%25E7%25A4%25BE%25E4%25BC%259A?topnav=1&wvr=6&b=1&display=0&retcode=6102'

In [205]:
driver = set_driver()

In [458]:
html_soup = get_feed_html_soup(driver, start_url)

In [442]:
mid_list = get_mid_list(html_soup)
print(mid_list)

['4346780374581298', '4346820799059078', '4346814767492117', '4346820765823538', '4346820740263682', '4346820619066894', '4346820580603113', '4346820576728532', '4346818282556028', '4346820547001044', '4346814524172163', '4346812465357444', '4346776142370038', '4346820358237685', '4346811118843330', '4346820274232950', '4346820266187894', '4346006906492611', '4346820157526028', '4346820039931262']


In [448]:
mid = mid_list[3]
print(mid)
account_link = get_account_link(html_soup, mid)
print(account_link)

4346820765823538


[<a class="name" href="//weibo.com/5479900504?refer_flag=1001030103_" nick-name="丷Joann丷" suda-data="key=tblog_search_weibo&amp;value=seqid:155184631147003269956|type:1|t:0|pos:1-0|q:%E7%A4%BE%E4%BC%9A|ext:cate:31,mpos:3,click:user_name" target="_blank">丷Joann丷</a>]
//weibo.com/5479900504?refer_flag=1001030103_


In [382]:
elems, elems_suda_data = get_comment_button_list(driver)

In [270]:
if comment != '0':
    click_comment(suda_data, elems, elems_suda_data)


In [344]:
html = driver.page_source.encode('utf-8')
html_soup = BeautifulSoup(html, "html.parser")

In [345]:
comment_id_list = get_comment_id_list(html_soup)
print(comment_id_list)

['4346795515714917', '4346795498638536', '4346794979119085', '4346794769239090', '4346794462286953', '4346794052157061', '4346793791798330', '4346793787848316', '4346793615330808']


In [352]:
comment_id = comment_id_list[0]
div_comment = html_soup.find_all('div', {'comment_id': comment_id})

In [353]:
comment_account_name = div_comment[0].find('div', {'class': 'txt'}).text.split()[0]
print(div_comment[0].find('div', {'class': 'txt'}).text.split())
#comment_text = div_comment[0].find('div', {'class': 'txt'}).text.split()[2]
#print(comment_account_name)
#print(comment_text)

['WOAIJIAJIA', '：']


In [307]:
"""
リストの結合
"""


A = [1, 2, 3]
B = [4, 5]

C = A + B
print(C)


[1, 2, 3, 4, 5]


In [308]:
C.clear()
print(C)

[]


In [33]:
feed_time = '3月4日 20:56'
feed_time = dt.strptime(feed_time, '%Y年%m月%d日%H:%M')

10


ValueError: time data '3月4日 20:56' does not match format '%Y年%m月%d日%H:%M'

In [37]:
text = '3月4日 20:56 转赞人数超过20'
text = text.split()[0] + text.split()[1]
# text = ''.join(text.split())
print(text)
feed_time = dt.strptime(text, '%m月%d日%H:%M').replace(year=2019)
print(feed_time)

3月4日20:56
2019-03-04 20:56:00


In [7]:
A = None

if A:
    print('true')
else:
    print('false')

false
